# Comfy Kettenregel (Autograd DIY) - univariate, skalare Funktionen

$$F(x) = f_1 \circ f_2 = f_1(f_2(x)) \Rightarrow f_1'(f_2(x)) \cdot f'_2(x)$$

$$F(x) = f_1 \circ f_2 \circ f_3 = f_1(f_2(f_3(x))) \Rightarrow f_1'(f_2(f_3(x))) \cdot f_2'(f_3
(x)) \cdot f_3'(x)$$

## Aufgabe

Ziel: Gradientenbasierte Optimierung von $f(x) = \sqrt{\frac{1}{e^{\sin(x)}}}$


### 0. Imports

In [ ]:
import math
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

### 1.0 Operationen definieren

In [ ]:
def one_div_x(x: float, inner_derivative: float = 1) -> tuple[float, float]:

    value = 1 / x
    derivative = -inner_derivative / x**2

    return value, derivative


def sin(x: float, inner_derivative: float = 1) -> tuple[float, float]:

    value = math.sin(x)
    derivative = math.cos(x) * inner_derivative

    return value, derivative


def sqrt(x: float, inner_derivative: float = 1) -> tuple[float, float]:

    value = math.sqrt(x)
    derivative = 1 / (2 * math.sqrt(x)) * inner_derivative

    return value, derivative


def exp(x: float, inner_derivative: float = 1) -> tuple[float, float]:

    value = math.exp(x)
    derivative = math.exp(x) * inner_derivative

    return value, derivative

### 1.1 Funktionsdefinition

In [ ]:
def f_x(x: float) -> tuple[float, float]:

    return sqrt(*one_div_x(*exp(*sin(x))))

### 2. Gradient Descent

In [ ]:
x_start = 4.0  # starting value
x_min = x_start - 8.0  # x-axis limits
x_max = x_start + 8.0
xs = []  # values for the animation
ys = []

lr = 1e-2  # step size
significant_gradient = 1e-3  # termination criteria
iter = 1  # counter

while True:
    y, deriv = f_x(x_start)
    if np.fabs(deriv) >= significant_gradient:
        xs.append(x_start)
        ys.append(y)
        x_start -= lr * deriv
        print(iter, x_start, y) if iter % 100 == 0 or iter == 1 else None
    else:
        xs.append(x_start)
        ys.append(y)
        break
    iter += 1

### 3.0 Funktionsplot

In [ ]:
x = np.arange(x_min, x_max, 0.01)

res = [f_x(_) for _ in x]
y, derivative = zip(*res)

df = pd.DataFrame(
    {
        "x": x,
        "y": y,
        "derivative": derivative,
    }
)

px.line(df, x="x", y="y")

### 3.1 Animation

In [ ]:
# get the values
x = np.arange(x_min, x_max, 0.01)

res = [f_x(_) for _ in x]
y, _ = zip(*res)

# define both graphs
fig = go.Figure(
    data=[
        go.Scatter(
            x=x,
            y=y,
            mode="lines",
            line=dict(color="green", width=1),
            name="Function Graph",
        ),
        go.Scatter(
            x=[xs[0]],
            y=[ys[0]],
            mode="markers",
            marker=dict(color="red", size=10),
            name="Current Position",
        ),
    ]
)

# update layout parameters and add start button for animation
fig.update_layout(
    width=1400,
    height=900,
    xaxis=dict(range=(x_min, x_max), autorange=False),
    yaxis=dict(range=(np.min(y) - 0.5, np.max(y) + 0.5), autorange=False),
    title_text="Gradient Descent Animation",
    # start button config
    updatemenus=[
        dict(
            type="buttons",
            buttons=[
                dict(
                    args=[
                        None,
                        {
                            "frame": {"duration": 5, "redraw": False},
                            "fromcurrent": True,
                            "transition": {"duration": 0, "easing": "linear"},
                        },
                    ],
                    label="start",
                    method="animate",
                )
            ],
        )
    ],
)

# specify the animation frames
fig.update(
    frames=[
        go.Frame(data=[go.Scatter(x=[xs[k]], y=[ys[k]])], traces=[1])
        for k in range(len(ys))
    ]
)

# show result
fig.show()